# Задание 1

С помощью библиотеки numpy реализовать модель с прямым проходом, состоящую из 3 полносвязных слоёв с функциями потерь: ReLU, tanh, Softmax. Длины векторов на входе 256, на выходе 4, промежуточные: 64 и 16.

In [20]:
import numpy as np
import random
import math

from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [21]:
def relu(x): #ReLU
    return max(0, x) 

def tanh_(x): #гиперболический тангенс
    return ((math.e**(2*x))-1)/(((math.e**(2*x))+1))

def softmax(mas): #софтмакс
    expa = np.exp(mas)
    return expa / np.sum(expa)

In [24]:
def first(mas256):
    #массив из n элементов, заполненный случайными числами в диапазоне [-1;1] 
    mas64 = np.array([random.uniform(-1,1) for i in range(64)])
    mas16 = np.array([random.uniform(-1,1) for i in range(16)])
    mas4 = np.array([random.uniform(-1,1) for i in range(4)]) 
    for i in range(64):#уменьшаем размер до 64
        for j in range(256):
            mas64[i] = mas64[i] + mas64[i] * mas256[j]
        
    mas64 = list(map(lambda x:relu(x), mas64)) #применяем ф.п. релу к элементам массива

    for i in range(16): #ещё уменьшаем..
        for j in range(64):
            mas16[i] = mas16[i] + mas16[i] * mas64[j]

    mas16 = list(map(lambda x:tanh_(x), mas16)) #применяем ф.п. гип. тангенса

    for i in range(4): #совсем уменьшаем.....
        for j in range(16):
            mas4[i] = mas4[i] + mas4[i] * mas16[j]

    mas4 = list(softmax(mas4)) #софтмакс
    return mas4#вот что получилось

    
    
    
mas256 = np.array([random.uniform(-1,1) for i in range(256)])
mas4 =first(mas256)    
print(np.sum(mas4)) #проверяем, сумма элеметов после софтмакса должна быть = 1

1.0


# Задание 2

Реализовать модель с прямым проходом, состоящую из 2 свёрток с функциями активации ReLU и 2 функций MaxPool. Первый слой переводит из 3 каналов в 8, второй из 8 слоёв в 16. На вход подаётся изображение размера 19х19. (19х19x3 -> 18x18x8 -> 9x9x8 -> 8x8x16 -> 4x4x16)

In [25]:
#свертка из 2ой лабы 2ой лекции
def matrix_conv(m1: list, m2: list):
    a, b = len(m1), len(m1[0])  
    c, d = len(m2), len(m2[0])
    res = [[0]*(b-d+1) for i in range(a-c+1)]
    for i in range(a-c+1):
        for j in range(b-d+1):
            tmp2 = 0
            for u in range(c):
                tmp1 = 0
                for v in range(d):
                    tmp1 += m1[i+u][j+v] * m2[u][v]
                tmp2 += tmp1
            res[i][j] = tmp2
    return res

In [26]:
def MP(mat):#maxpooling
    sh = list(mat.shape)
    sh[1], sh[2] = [sh[1] // 2] * 2
    mpres = np.zeros(tuple(sh))
    
    for d in range(sh[0]):
        s = 0
        for i in range (sh[1]):
            for j in range (sh[2]):
                mpres[d][i][j] = max(mat[d][i+s][j*2], mat[d][i+s+1][j*2], mat[d][i+s][j*2+1], mat[d][i+s+1][j*2+1])
            s += 1
    return mpres

In [27]:
def sec(pic):
    layer8 = np.zeros ((8,18,18))

    for i in range (8):
        conv = np.random.uniform(low = -1, high = 1, size =(3,2,2)) # каждый раз генерим новое ядро для свёртки
    
        layer3 = np.array([matrix_conv(pic[j], conv[j]) for j in range (3)]) #свёрточка 3 раза, тк 3 потока

        layer3 = layer3[0] + layer3[1] + layer3[2] 
        layer3 = layer3.reshape(18*18) #в 1D
        layer3 = np.array(list(map(lambda x:relu(x), layer3))) #ReLU
        layer8[i] = layer3.reshape(18,18) #в 2D

    mp1 = MP(layer8)#maxpooling
    layer16 = np.zeros((16,8,8))

    for a in range (16): #повторяем
        conv = np.random.uniform(low = -1, high = 1, size =(8,2,2))
    
        mp2 = np.array([matrix_conv(mp1[k], conv[k]) for k in range (8)])
        layer8 = np.zeros((8,8))
        for t in range (8):
            layer8 += mp2[t]
        layer8 = layer8.reshape(8*8)
        layer8 = np.array(list(map(lambda a:relu(a), layer8)))
        layer16[a] = layer8.reshape(8,8)

    answ = MP(layer16)
    return answ

    
pic = np.random.uniform(low = -1, high = 1, size=(3,19,19))
sec(pic)

array([[[ 6.18279593,  5.87640812,  5.11118395,  5.65784279],
        [ 7.14580548,  2.67379416,  8.23488662,  7.49904832],
        [ 9.58074398,  5.89630508,  4.72410133,  8.13678947],
        [ 1.40048224,  6.62146233,  7.33776287,  3.80836484]],

       [[ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.28785631],
        [ 0.        ,  0.        ,  0.        ,  1.04459246],
        [ 0.        ,  0.        ,  0.        ,  0.        ]],

       [[ 2.0687486 ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.17227006,  2.95150787,  0.        ],
        [ 0.        ,  0.        ,  2.16080772,  0.        ]],

       [[ 7.30773001,  5.87325753,  2.2456509 ,  6.54985039],
        [ 6.29792572,  4.78319788,  5.42656144,  5.26316044],
        [ 4.59603967,  4.06931474,  8.32057026,  3.1634134 ],
        [ 5.99886624,  5.69839242,  5.57903901,  3.18707202]],



# Задание 3

Объединить сети из п.2 и п.1. На вход изображение размера 19х19, на выходе вектор из 4 элементов

In [29]:
picture = np.random.uniform(low = -50,high = 50,size = (3,19,19))
pic = sec(picture)

pic = pic.reshape(16 * 4 * 4)
pic = first(pic)

print(pic)

[0.25708490013927365, 0.24719304676717227, 0.2369328998842636, 0.25878915320929036]


C:\Users\79132\AppData\Local\Temp\ipykernel_10348\757225287.py:8: RuntimeWarning: overflow encountered in double_scalars
  mas64[i] = mas64[i] + mas64[i] * mas256[j]
C:\Users\79132\AppData\Local\Temp\ipykernel_10348\757225287.py:8: RuntimeWarning: invalid value encountered in double_scalars
  mas64[i] = mas64[i] + mas64[i] * mas256[j]


# Лабораторная работа 10. Матчасть DL

In [30]:
def sigmoid (x):
    return 1/(1 + np.exp(-x))
    

In [31]:
class Neuron:
    def __init__ (self, inputLayerNeurons: int, hiddenLayerNeurons: int):
        self.weights = np.random.uniform(size=(inputLayerNeurons,hiddenLayerNeurons))
        self.bias =np.random.uniform(size=(1,hiddenLayerNeurons))
        self.output = []
    
    def forward(self, inputs):
        hidden_layer_activation = np.dot(inputs,self.weights)
        self.output = sigmoid(hidden_layer_activation)
        return self.output
    
    def backward2(self, inputs, loss):
        self.weights += np.dot(inputs, loss)
        self.bias += np.sum(loss, axis = 0, keepdims=True)
        
    def sigmoid_derivative(x):
        return x * (1 - x)
        
class Model:
    def __init__(self, input_layers:int, hidden_layers:int, output_layers:int):
        self.hidden_layers = Neuron(input_layers, hidden_layers)
        self.output_layers = Neuron(hidden_layers, output_layers)
        self.output = 0
        
    def forward(self, inputs):
        hidden_output = self.hidden_layers.forward(inputs)
        self.output = self.output_layers.forward(hidden_output)
        return self.output
    
    def backward(self, input_, expected_output, predicted_output):
        error = expected_output - predicted_output
        d_predicted_output = error * Neuron.sigmoid_derivative(predicted_output)
        
        error_hidden_layers = d_predicted_output.dot(self.output_layers.weights.T)
        d_hidden_layer = error_hidden_layers * Neuron.sigmoid_derivative(self.hidden_layers.output)
        
        self.output_layers.backward2(self.hidden_layers.output.T, d_predicted_output)
        self.hidden_layers.backward2(input_.T, d_hidden_layer)
    
    def train(self, X, y, epochs):
        for i in range(epochs):
            output = self.forward(X)
            self.backward(X,y,output)

    def predict(self,X):
        return self.forward(X)
    
    def accuracy(self, y, predict):
        return 1 - (np.abs(y-predict))


In [32]:
x = np.array([[0,0],[0,1],[1,0],[1,1]])
y = np.array([[0],[1],[1],[0]])

input_size = 2
hidden_size = 2
output_size = 1

model = Model(input_size, hidden_size, output_size)
model.train(x, y, 10000)
output = model.predict([[0,0],[1,0]])

print (output, model.accuracy([[0],[1]], output))

[[0.03227753]
 [0.93162327]] [[0.96772247]
 [0.93162327]]
